In [1]:
from qa_helper import *

In [2]:
init_kg()

In [3]:
history = read_json('../data/history.json')


questions = ['when does the summer term start?', 'how many credits do I need to graduate?', 'when is the convocation held', 'how many online courses can I do in a semester?']
for question in questions:
    print('-' * 60)
    print(question)
    
    answer = []
    sentences = shortlist_sentences(question)

    for s in range(len(sentences)):
        print(s, ':', sentences[s]['sentence'])
    continue
    
    if len(sentences) == 0:
    	print('-' * 60)
    	print('Sorry, seems like we didn\'t find anything related to this question!')
    	history[question] = None

    elif len(sentences) <= 3:
    		answer = sentences

    if not answer:
        print('-' * 60)
        print('Please wait...')
        answer = find_answer_from_bert(question, sentences)
        # answer = find_answer_from_mrc(question, sentences)

    print('-' * 60)
    for i in range(len(answer)):
    	print(i + 1, answer[i]['sentence'])

------------------------------------------------------------
when does the summer term start?
[['summer semester', ['semester'], ['summer', 'term', 'start']]]
['datetime']


TypeError: unsupported operand type(s) for /: 'list' and 'int'